Please save a copy of the script to your drive to use it :) 
If you have any issues feel free to email me ruth.everett@deepcrawl.com or on Twitter @rvtheverett. 

To use run each cell at a time and upload your csv file when prompted. The first two steps are optional for testing the packages are installed, the API call is set up correctly and the results are able to be printed. 

**PLEASE NOTE:** As this is using the API  I would recommend just using a small set of URLs at a time, especially if you have a large site. For example, you could use your highest trafficked pages, or pages which generate the most revenue. Alternatively if you use templates it would be great for testing sets of these.

Step 1: Import required packages


In [ ]:
# Import required packages 
import json
import requests
import pandas as pd
import urllib
import time
from google.colab import files
import io

**Optional Step:** Running one URL through the API to test 

In [ ]:
# Define URL  
url = 'https://www.asos.com/women/'

# API request url
result = urllib.request.urlopen('https://www.googleapis.com/pagespeedonline/v5/runPagespeed?url={}/&strategy=mobile'\
.format(url)).read().decode('UTF-8')
print(result)


**Optional Step:** Converting to JSON format to download and analyse for single URL



In [ ]:
# Convert to json format
result_json = json.loads(result)

print(result_json)

with open('result.json', 'w') as outfile:
  json.dump(result_json, outfile)

files.download('result.json')

Upload a csv file of URLs, ensure the column header is titled 'url' (if not, please define the column header as the variable where instructed)




In [ ]:
uploaded = files.upload()
#if your column header is something other than 'url' please define it here 
column_header='url'


Saving urls.csv to urls.csv


Turning csv file into a Pandas dataframe and reviewing 


In [ ]:
# Get the filename from the upload so we can read it into a CSV.
for key in uploaded.keys():
  filename = key
# Read the selected file into a Pandas Dataframe
df = pd.read_csv(io.BytesIO(uploaded[filename]))

df.head()
 

Iterate through the list of URLs to run through the Page Speed API *(please note this step may take some time to complete, as we have added a 30 second break between URLs to prevent overloading the API)*


In [ ]:
response_object = {}


# Iterate through the df
for x in range(0, len(df)):

        # Define request parameter
        url = df.iloc[x][column_header]

        # Make request
        pagespeed_results = urllib.request.urlopen('https://www.googleapis.com/pagespeedonline/v5/runPagespeed?url={}&strategy=mobile'\
            .format(url)).read().decode('UTF-8')

        # Convert to json format
        pagespeed_results_json = json.loads(pagespeed_results)

        # Insert returned json response into response_object
        response_object[url] = pagespeed_results_json
        time.sleep(30)
        
        print(response_object[url])

Create dataframe to store responses


In [ ]:
# Create dataframe to store field data responses
df_core_web_vitals = pd.DataFrame(columns=['url',
          'Overall_Category',
          'Largest_Contentful_Paint_Category',
          'Largest_Contentful_Paint_Percentile',
          'First_Input_Delay_Category',
          'First_Input_Delay_Percentile',
          'Cumulative_Layout_Shift_Category',
          'Cumulative_Layout_Shift_Percentile'       
          ])  

print(df_core_web_vitals)

Iterate through the response object to pull out desired metrics


In [ ]:
for (url, x) in zip(
    response_object.keys(),
    range(0, len(response_object))
):

        # URLs
        df_core_web_vitals.loc[x, 'url'] =\
            response_object[url]['lighthouseResult']['finalUrl']

        # Overall Category
        df_core_web_vitals.loc[x, 'Overall_Category'] =\
            response_object[url]['loadingExperience']['overall_category']   

        # Core Web Vitals     

        # Largest Contentful Paint 
        lcp = response_object[url]\
            ['loadingExperience']['metrics']['LARGEST_CONTENTFUL_PAINT_MS']

        # First Input Delay 
        fid = response_object[url]\
            ['loadingExperience']['metrics']['FIRST_INPUT_DELAY_MS']

        # Cumulative Layout Shift
        cls = response_object[url]\
            ['loadingExperience']['metrics']['CUMULATIVE_LAYOUT_SHIFT_SCORE']

        # First Contentful Paint Metrics     
        df_core_web_vitals.loc[x, 'Largest_Contentful_Paint_Category'] = lcp['category']
        df_core_web_vitals.loc[x, 'Largest_Contentful_Paint_Percentile'] = lcp['percentile']

        # First Input Delay Metrics     
        df_core_web_vitals.loc[x, 'First_Input_Delay_Category'] = fid['category']
        df_core_web_vitals.loc[x, 'First_Input_Delay_Percentile'] = fid['percentile']

        # Cumulative Layout Shift Metrics     
        df_core_web_vitals.loc[x, 'Cumulative_Layout_Shift_Category'] = cls['category']
        df_core_web_vitals.loc[x, 'Cumulative_Layout_Shift_Percentile'] = cls['percentile']

Create a summary dataframe of the results collected 




In [ ]:
summary = df_core_web_vitals

df_core_web_vitals.head()

Download the csv file


In [ ]:
#Download csv file 
summary.to_csv('core_web_vitals.csv')
files.download('core_web_vitals.csv')